<a href="https://colab.research.google.com/github/AtomZa/BadApple-EdgeDetection/blob/main/EasyAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Example Template for P'Friend

Feature: Accept multiple time series input data 
 Ex. [(0.5, 0.6, 0.5), (0.4, 0.2, 0.1), (0.2, 0.1, 0.3)] labeling with class [0] or [1].

*Note that time series size and number of classes can be adjust later.*

After I recieving the dataset I can build the dataloader framework for you krub :)

## Deep Learning Algorithms
1. 1D CNN
2. LSTM

In [244]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# 1D CNN

In [245]:
# Define the 1D CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=2, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=16, out_features=2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.softmax(x)
        return x

# Instantiate the model and define the loss function and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [246]:
# Define the dataset
X = np.array([[0.5, 0.6, 0.5], [0.4, 0.2, 0.1], [0.2, 0.1, 0.3]], dtype=np.float32)
y = np.array([1, 0, 1], dtype=np.int64)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).unsqueeze(1)  # Add a channel dimension to the data
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test).unsqueeze(1)  # Add a channel dimension to the data
y_test = torch.from_numpy(y_test)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
# Evaluate the model on the test set
with torch.no_grad():
    test_output = model(X_test)
    test_loss = criterion(test_output, y_test)
    test_pred_classes = torch.argmax(test_output, dim=1)
    test_accuracy = torch.mean((test_pred_classes == y_test).float())
    print('Test Accuracy: {:.2f}%'.format(test_accuracy.item()*100))


Epoch [10/100], Loss: 0.7090
Epoch [20/100], Loss: 0.7084
Epoch [30/100], Loss: 0.7078
Epoch [40/100], Loss: 0.7072
Epoch [50/100], Loss: 0.7067
Epoch [60/100], Loss: 0.7061
Epoch [70/100], Loss: 0.7056
Epoch [80/100], Loss: 0.7051
Epoch [90/100], Loss: 0.7046
Epoch [100/100], Loss: 0.7041
Test Accuracy: 100.00%


# LSTM

In [247]:
# define the LSTM class
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        
        # Propagate input through LSTM
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Pass the output of the last time step through the fully connected layer
        out = self.fc(out[:, -1, :])
        return out

# set the hyperparameters of the model
input_size = 3
hidden_size = 4
num_layers = 1
output_size = 2

# create an instance of the LSTM class
model = LSTM(input_size, hidden_size, num_layers, output_size)

# set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [248]:
# define the input data and corresponding labels
input_data = np.array([(0.5, 0.6, 0.5), (0.4, 0.2, 0.1), (0.2, 0.1, 0.3)], dtype=np.float32)
labels = np.array([1, 0, 1], dtype=np.int64)

# split the input data and labels into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(input_data, labels, test_size=0.2)

# create PyTorch tensors from the input data and labels
x_train = torch.from_numpy(x_train).unsqueeze(0)
y_train = torch.from_numpy(y_train)

x_test = torch.from_numpy(x_test).unsqueeze(0)
y_test = torch.from_numpy(y_test)

# train the model
num_epochs = 1000
for epoch in range(num_epochs):
    # randomly sample a batch of data
    indices = torch.randperm(x_train.size(0))
    batch_x, batch_y = x_train[indices], y_train[indices]
    
    optimizer.zero_grad()
    output = model(batch_x)
    loss = criterion(output, batch_y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

# test the model
with torch.no_grad():
    output = model(x_test)
    _, predicted = torch.max(output.data, 1)
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    accuracy = correct/total
    print("Test Accuracy: {:.2f}%".format(accuracy*100))



Epoch [1/1000], Loss: 0.8210
Epoch [101/1000], Loss: 0.0077
Epoch [201/1000], Loss: 0.0031
Epoch [301/1000], Loss: 0.0018
Epoch [401/1000], Loss: 0.0012
Epoch [501/1000], Loss: 0.0008
Epoch [601/1000], Loss: 0.0006
Epoch [701/1000], Loss: 0.0005
Epoch [801/1000], Loss: 0.0004
Epoch [901/1000], Loss: 0.0003
Test Accuracy: 100.00%
